In [7]:
import pandas as pd
import time
import random
import math
import mmh3

# Fairness na etapa de Blocagem

In [ ]:
# pd.set_option('display.max_columns', 67)

In [ ]:
#[('BZ58220', 'F'), ('BZ5995', 'M'), ('BZ75127', 'M'), ('BZ71367', 'F'), ('BZ75022', 'F'), ('BZ75023', 'F'), ('BZ39687', 'M'), ('BZ57774', 'M'), ('EH487995', 'F'), ('BZ65623', 'M'), ('BZ7054', 'F'), ('BZ7042', 'M'), ('BZ61329', 'M'), ('BZ71427', 'F'), ('BZ14188', 'F'), ('BZ4312', 'F'), ('BZ61767', 'F'), ('BZ4215', 'M'), ('BZ69560', 'F'), ('BZ65176', 'F'), ('EH1023266', 'F'), ('BZ77673', 'M'), ('BZ9596', 'M')]
def calcula_qtd_genero_por_bloco(bloco):
    """
    Calcula a quantidade de generos(masculino e feminino) de um bloco específico.
    """
    masculino=0
    feminino=0
    for id, genero in bloco:
        if genero == 'M':
            masculino += 1
        elif genero == 'F':
            feminino +=1
    return {"masculino": masculino, "feminino": feminino}

In [9]:
def str_to_MinHash(str1, q, seed=0):
    return min([mmh3.hash(str1[i:i + q], seed) for i in range(len(str1) - q + 1)])

def frequent2(temp, L, t):
    return {k: v for (k, v) in temp.items() if v/L >= t}

def matching():
    global tp, fp, pairsNo, L1, q
    for index2 in range(nbS, nbS + offsetB): 
        if index2 > len(df2) - 1:
            return True

        rr = df2.iloc[index1]
        ncid = rr["id"]
        first_name = rr["first_name"]
        last_name = rr["last_name"]
        registr_dt = rr["registr_dt"]
        age_at_year_end = rr["age_at_year_end"]
        gender_code = rr["gender_code"]
        srec = first_name + " " + last_name + " " + str(registr_dt) + " " + str(age_at_year_end)

        temp = dict()
        indices = [random.randrange(0, L) for i in range(L1)]
        matchingPairs = {}
        for l in indices:
            key = str(str_to_MinHash(srec.lower(), q, l))
            d = dictB[l]
            if key in d:
                ids = d[key]
                # print(ids)
                for id, gender in ids:
                    if id in temp:
                        temp[id] += 1
                        if temp[id] / L1 >= t:
                            matchingPairs[id] = 1
                    else:
                        temp[id] = 1
        for id in matchingPairs.keys():
            idDBLP = id
            pairsNo += 1
            if idDBLP in truthD:
                ids = truthD[idDBLP]
                for id in ids:
                    if id == ncid:
                        tp += 1
                        break
            else:
                fp += 1

    return False

def elimina_elementos_dentro_dictB(array_para_descarte, array_para_descarte_igual):
    array_para_descarte.clear()
    array_para_descarte_igual.clear()

In [10]:
df1 = pd.read_csv("../00-datasets/ncvoter42.csv", sep=",", encoding="utf-8", keep_default_na=False)
df2 = pd.read_csv("../00-datasets/perturbed_recordsNC_voter.csv", sep=",", encoding="utf-8", keep_default_na=False)
truth = pd.read_csv("../00-datasets/ground_truthNC_voter.csv", sep=",", encoding="utf-8", keep_default_na=False)

truthD = dict()
for i, r in truth.iterrows():
    novoId = r["novoId"]
    antigoId = r["antigoId"]
    if antigoId in truthD:
        ids = truthD[antigoId]
        ids.append(novoId)
    else:
        truthD[antigoId] = [novoId]

df1 = df1[df1["gender_code"] != "U"]
df2 = df2[df2["gender_code"] != "U"]

t = 0.5
TP = 40893
eps = 0.1
w = 1000
delta = 0.1
L = math.ceil(math.log(1 / delta) / (2 * (eps ** 2)))
eps = 0.01
L1 = int(1 / (2 * eps))
print("L=", L, "L1=", L1)
q = 2


L= 116 L1= 50


In [11]:
def adiciona_valor_bloco(ids, ids_igual, gender_code , tempoQueFoiInseridoNaEstrutura, ncid):
    resultado = calcula_qtd_genero_por_bloco(ids)
    if resultado["masculino"] > resultado["feminino"] and gender_code == "M":
        # print("Tem mais homem aqui e vai adicionar mais outro homem")
        pass
    elif resultado["feminino"] > resultado["masculino"] and gender_code == "F":
        # print("Tem mais mulher aqui e vai adicionar mais outra mulher")
        pass
    elif resultado["masculino"] > resultado["feminino"] and gender_code == "F":
        ids.append((ncid, gender_code))
        ids_igual.append(tempoQueFoiInseridoNaEstrutura)
    elif resultado["feminino"] > resultado["masculino"] and gender_code == "M":
        ids.append((ncid, gender_code))
        ids_igual.append(tempoQueFoiInseridoNaEstrutura)
    else:
        ids.append((ncid, gender_code))
        ids_igual.append(tempoQueFoiInseridoNaEstrutura)      

In [12]:
tp = 0
fp = 0
pairsNo = 0
nbS = 1
naS = 1
offsetA = 50
offsetB = 50
blockingTime = 0
matchingTime = 0
dictB = [dict() for l in range(L)]
dictB_igual = [dict() for l in range(L)]

tempoQueFoiInseridoNaEstrutura = 0 # esse é o valor que sera colocado junto com o id, seria a posiçaõ do elemento no array

tamanhoDosBlocos = {}

while True:
    st = time.time()
    for index1 in range(naS, naS + offsetA):
        if index1 >= len(df1):
            break
        rr = df1.iloc[index1, 0:67]
        ncid = rr["ncid"]
        first_name = rr["first_name"]
        last_name = rr["last_name"]
        registr_dt = rr["registr_dt"]
        age_at_year_end = rr["age_at_year_end"]
        gender_code = rr["gender_code"]
        srec = first_name + " " + last_name + " " + str(registr_dt) + " " + str(age_at_year_end)
        key = ""
        
        for l in range(L):
            key = str(str_to_MinHash(srec.lower(), 2, l))
            d = dictB[l]
            d_igual = dictB_igual[l]
            if key in d:
                ids = d[key]
                ids_igual = d_igual[key]
                tamanho_atual = tamanhoDosBlocos[(key, l)]
                if len(ids) < tamanho_atual:
                    adiciona_valor_bloco(ids,ids_igual, gender_code, tempoQueFoiInseridoNaEstrutura, ncid)
                else:
                    elimina_elementos_dentro_dictB(ids, ids_igual)

                    adiciona_valor_bloco(ids,ids_igual, gender_code, tempoQueFoiInseridoNaEstrutura, ncid)
            else:
                d[key] = [(ncid, gender_code)]
                d_igual[key] = [tempoQueFoiInseridoNaEstrutura]

                tamanhoDosBlocos[(key, l)] = w
            
        tempoQueFoiInseridoNaEstrutura += 1
    
    end = time.time()

    blockingTime += (end - st)
    st = time.time()
    termination = matching()  
    end = time.time()
    matchingTime += (end - st)
    if termination:
            break

    nbS += offsetB
    naS += offsetA

print("blocking time (in mins)", blockingTime / 60)
print("matching time (in mins)", matchingTime / 60)
if tp + fp > 0:
    print("TP=", tp, "Recall=", tp / TP, "Precision=", tp / (tp + fp), "pairsNo=", pairsNo)

blocking time (in mins) 3.818868947029114
matching time (in mins) 2.328447083632151
TP= 24085 Recall= 0.5889761083804074 Precision= 0.7181620299967201 pairsNo= 42234


# Fairness na etapa de Blocagem, com Contador para melhorar Velocidade

In [37]:
def str_to_MinHash(str1, q, seed=0):
    return min([mmh3.hash(str1[i:i + q], seed) for i in range(len(str1) - q + 1)])

def frequent2(temp, L, t):
    return {k: v for (k, v) in temp.items() if v/L >= t}

def matching():
    global tp, fp, pairsNo, L1, q
    for index2 in range(nbS, nbS + offsetB): 
        if index2 > len(df2) - 1:
            return True

        rr = df2.iloc[index1]
        ncid = rr["id"]
        first_name = rr["first_name"]
        last_name = rr["last_name"]
        registr_dt = rr["registr_dt"]
        age_at_year_end = rr["age_at_year_end"]
        gender_code = rr["gender_code"]
        srec = first_name + " " + last_name + " " + str(registr_dt) + " " + str(age_at_year_end)

        temp = dict()
        indices = [random.randrange(0, L) for i in range(L1)]
        matchingPairs = {}
        for l in indices:
            key = str(str_to_MinHash(srec.lower(), q, l))
            d = dictB[l]
            if key in d:
                ids = d[key]
                # print(ids)
                for id, gender in ids:
                    if id in temp:
                        temp[id] += 1
                        if temp[id] / L1 >= t:
                            matchingPairs[id] = 1
                    else:
                        temp[id] = 1
        for id in matchingPairs.keys():
            idDBLP = id
            pairsNo += 1
            if idDBLP in truthD:
                ids = truthD[idDBLP]
                for id in ids:
                    if id == ncid:
                        tp += 1
                        break
            else:
                fp += 1

    return False

def elimina_elementos_dentro_dictB(array_para_descarte, array_para_descarte_igual, array_genero_para_descarte):
    array_para_descarte.clear()
    array_para_descarte_igual.clear()
    array_genero_para_descarte["masculino"] = 0
    array_genero_para_descarte["feminino"] = 0

In [38]:
df1 = pd.read_csv("../00-datasets/ncvoter42.csv", sep=",", encoding="utf-8", keep_default_na=False)
df2 = pd.read_csv("../00-datasets/perturbed_recordsNC_voter.csv", sep=",", encoding="utf-8", keep_default_na=False)
truth = pd.read_csv("../00-datasets/ground_truthNC_voter.csv", sep=",", encoding="utf-8", keep_default_na=False)

truthD = dict()
for i, r in truth.iterrows():
    novoId = r["novoId"]
    antigoId = r["antigoId"]
    if antigoId in truthD:
        ids = truthD[antigoId]
        ids.append(novoId)
    else:
        truthD[antigoId] = [novoId]

df1 = df1[df1["gender_code"] != "U"]
df2 = df2[df2["gender_code"] != "U"]

t = 0.5
TP = 40893
eps = 0.1
w = 1000
delta = 0.1
L = math.ceil(math.log(1 / delta) / (2 * (eps ** 2)))
eps = 0.01
L1 = int(1 / (2 * eps))
print("L=", L, "L1=", L1)
q = 2


L= 116 L1= 50


In [39]:
def adiciona_valor_bloco(ids, ids_igual, ids_genero, gender_code , tempoQueFoiInseridoNaEstrutura, ncid):
    # ter de alguma maneira a qtd de masculino e feminino dentro do bloco atraves de variaveis
    # resultado = calcula_qtd_genero_por_bloco(ids)
    # print("Antes de adicionar")
    # print(ids)
    # print(ids_igual)
    # print(ids_genero["masculino"])
    # print("\n\n")
    
    if ids_genero["masculino"] > ids_genero["feminino"] and gender_code == "M":
        # print("Tem mais homem aqui e vai adicionar mais outro homem")
        pass
    elif ids_genero["feminino"] > ids_genero["masculino"] and gender_code == "F":
        # print("Tem mais mulher aqui e vai adicionar mais outra mulher")
        pass
    elif ids_genero["masculino"] > ids_genero["feminino"] and gender_code == "F":
        ids.append((ncid, gender_code))
        ids_igual.append(tempoQueFoiInseridoNaEstrutura)
        ids_genero["feminino"] += 1
    elif ids_genero["feminino"] > ids_genero["masculino"] and gender_code == "M":
        ids.append((ncid, gender_code))
        ids_igual.append(tempoQueFoiInseridoNaEstrutura)
        ids_genero["masculino"] += 1
    else:
        ids.append((ncid, gender_code))
        ids_igual.append(tempoQueFoiInseridoNaEstrutura) 
        if gender_code == "M":
            ids_genero["masculino"] += 1
        else:
            ids_genero["feminino"] += 1

    # print("Depois de adicionar")
    # print(ids)
    # print(ids_igual)
    # print(ids_genero)
    # print("\n\n")
    

In [40]:
tp = 0
fp = 0
pairsNo = 0
nbS = 1
naS = 1
offsetA = 50
offsetB = 50
blockingTime = 0
matchingTime = 0
dictB = [dict() for l in range(L)]
dictB_igual = [dict() for l in range(L)]
dictB_genero = [dict() for l in range(L)] # esse dicionario é para guardar a qtd de masculino e feminino dentro do bloco    

tempoQueFoiInseridoNaEstrutura = 0 # esse é o valor que sera colocado junto com o id, seria a posiçaõ do elemento no array

tamanhoDosBlocos = {}

while True:
    st = time.time()
    for index1 in range(naS, naS + offsetA):
        if index1 >= len(df1):
            break
        rr = df1.iloc[index1, 0:67]
        ncid = rr["ncid"]
        first_name = rr["first_name"]
        last_name = rr["last_name"]
        registr_dt = rr["registr_dt"]
        age_at_year_end = rr["age_at_year_end"]
        gender_code = rr["gender_code"]
        srec = first_name + " " + last_name + " " + str(registr_dt) + " " + str(age_at_year_end)
        key = ""
        
        for l in range(L):
            key = str(str_to_MinHash(srec.lower(), 2, l))
            d = dictB[l]
            d_igual = dictB_igual[l]
            d_genero = dictB_genero[l]  # dicionario para guardar a qtd de masculino e feminino dentro do bloco
            if key in d:
                ids = d[key]
                ids_igual = d_igual[key]
                ids_genero = d_genero[key]  # pegar a qtd de masculino e feminino dentro do bloco

                tamanho_atual = tamanhoDosBlocos[(key, l)]
                if len(ids) < tamanho_atual:
                    
                    adiciona_valor_bloco(ids,ids_igual,ids_genero, gender_code, tempoQueFoiInseridoNaEstrutura, ncid)
                else:
                    elimina_elementos_dentro_dictB(ids, ids_igual, ids_genero)

                    adiciona_valor_bloco(ids,ids_igual, ids_genero, gender_code, tempoQueFoiInseridoNaEstrutura, ncid)
            else:
                d[key] = [(ncid, gender_code)]
                d_igual[key] = [tempoQueFoiInseridoNaEstrutura]
                d_genero[key] = {"masculino": 1 if gender_code == "M" else 0, "feminino": 1 if gender_code == "F" else 0}
                # print(d_genero[key])
                tamanhoDosBlocos[(key, l)] = w
            
        tempoQueFoiInseridoNaEstrutura += 1
    
    end = time.time()

    blockingTime += (end - st)
    st = time.time()
    termination = matching()  
    end = time.time()
    matchingTime += (end - st)
    if termination:
            break

    nbS += offsetB
    naS += offsetA

print("blocking time (in mins)", blockingTime / 60)
print("matching time (in mins)", matchingTime / 60)
if tp + fp > 0:
    print("TP=", tp, "Recall=", tp / TP, "Precision=", tp / (tp + fp), "pairsNo=", pairsNo)

blocking time (in mins) 0.7031747341156006
matching time (in mins) 2.5422427972157795
TP= 24132 Recall= 0.5901254493434084 Precision= 0.712362734679419 pairsNo= 42838


# Fairness na etapa de Matching

In [56]:
def calcula_diferença_m_f(equidade_matching):
    valor = 0
    if equidade_matching["M"] > equidade_matching["F"]:
        # print("tem que matching Feminino")
        valor = -1
    elif equidade_matching["F"] > equidade_matching["M"]:
        # print("tem que matching Masculino")
        valor = 1
    else:
        # print("pode ser os dois")
        valor = 0
    return valor

In [ ]:
def str_to_MinHash(str1, q, seed=0):
    return min([mmh3.hash(str1[i:i + q], seed) for i in range(len(str1) - q + 1)])

def frequent2(temp, L, t):
    return {k: v for (k, v) in temp.items() if v/L >= t}

def matching():
    global tp, fp, pairsNo, L1, q, equidade_matching
    for index2 in range(nbS, nbS + offsetB): 
        if index2 > len(df2) - 1:
            return True

        rr = df2.iloc[index1]
        ncid = rr["id"]
        first_name = rr["first_name"]
        last_name = rr["last_name"]
        registr_dt = rr["registr_dt"]
        age_at_year_end = rr["age_at_year_end"]
        gender_code = rr["gender_code"]
        srec = first_name + " " + last_name + " " + str(registr_dt) + " " + str(age_at_year_end)

        temp = dict()
        indices = [random.randrange(0, L) for i in range(L1)]
        matchingPairs = {}
        for l in indices:
            key = str(str_to_MinHash(srec.lower(), q, l))
            d = dictB[l]
            if key in d:
                ids = d[key]
                for id, genderss in ids:
                    
                    if id in temp:
                        temp[id] += 1
                        valor = calcula_diferença_m_f(equidade_matching) # se for -1, esse genero tem que ser feminino, 
                                                                        # se for 1, tem que ser masculino, se for 0 pode ser qualquer 1 dos dois                                            
                        if (temp[id] / L1 >= t) and ((valor == -1 and genderss == "F") or (valor == 1 and genderss == "M") or (valor == 0)):
                            matchingPairs[id] = 1
                            equidade_matching[genderss] += 1
                            # print(equidade_matching)
                    else:
                        temp[id] = 1

        for id in matchingPairs.keys():
            idDBLP = id
            pairsNo += 1
            if idDBLP in truthD:
                ids = truthD[idDBLP]
                for id in ids:
                    if id == ncid:
                        tp += 1
                        break
            else:
                fp += 1

    return False


def elimina_elementos_dentro_dictB(array_para_descarte, array_para_descarte_igual):
    array_para_descarte.clear()
    array_para_descarte_igual.clear()

In [61]:
df1 = pd.read_csv("../00-datasets/ncvoter42.csv", sep=",", encoding="utf-8", keep_default_na=False)
df2 = pd.read_csv("../00-datasets/perturbed_recordsNC_voter.csv", sep=",", encoding="utf-8", keep_default_na=False)
truth = pd.read_csv("../00-datasets/ground_truthNC_voter.csv", sep=",", encoding="utf-8", keep_default_na=False)
truthD = dict()
for i, r in truth.iterrows():
    novoId = r["novoId"]
    antigoId = r["antigoId"]
    if antigoId in truthD:
        ids = truthD[antigoId]
        ids.append(novoId)
    else:
        truthD[antigoId] = [novoId]

df1 = df1[df1["gender_code"] != "U"]
df2 = df2[df2["gender_code"] != "U"]

t = 0.5
TP = 40893
eps = 0.1
w = 1000
delta = 0.1
L = math.ceil(math.log(1 / delta) / (2 * (eps ** 2)))
eps = 0.01
L1 = int(1 / (2 * eps))
print("L=", L, "L1=", L1)
q = 2

L= 116 L1= 50


In [62]:
tp = 0
fp = 0
pairsNo = 0
nbS = 1
naS = 1
offsetA = 50
offsetB = 50
blockingTime = 0
matchingTime = 0
tempoQueFoiInseridoNaEstrutura = 0 # esse é o valor que sera colocado junto com o id, seria a posiçaõ do elemento no array
tamanhoDosBlocos = {}
equidade_matching = {"M": 0, "F": 0} # esse dicionario é para guardar a qtd de masculino e feminino que foram encontrados no matching

dictB = [dict() for l in range(L)]
dictB_igual = [dict() for l in range(L)]

while True:
    st = time.time()
    for index1 in range(naS, naS + offsetA):
        if index1 >= len(df1):
            break
        rr = df1.iloc[index1, 0:69]
        ncid = rr["ncid"]
        first_name = rr["first_name"]
        last_name = rr["last_name"]
        registr_dt = rr["registr_dt"]
        age_at_year_end = rr["age_at_year_end"]
        gender_code = rr["gender_code"]
        srec = first_name + " " + last_name + " " + str(registr_dt) + " " + str(age_at_year_end)
        key = ""
        
        for l in range(L):
            key = str(str_to_MinHash(srec.lower(), 2, l))
            d = dictB[l]
            d_igual = dictB_igual[l]
            if key in d:
                ids = d[key]
                ids_igual = d_igual[key]
                tamanho_atual = tamanhoDosBlocos[(key, l)]
                if len(ids) < tamanho_atual:
                    ids.append((ncid, gender_code))
                    ids_igual.append(tempoQueFoiInseridoNaEstrutura)
                else:
                    elimina_elementos_dentro_dictB(ids, ids_igual)

                    ids.append((ncid, gender_code))
                    ids_igual.append(tempoQueFoiInseridoNaEstrutura)
            else:
                d[key] = [(ncid, gender_code)]
                d_igual[key] = [tempoQueFoiInseridoNaEstrutura]

                tamanhoDosBlocos[(key, l)] = w
            
        tempoQueFoiInseridoNaEstrutura += 1
    
    end = time.time()

    blockingTime += (end - st)
    st = time.time()
    termination = matching()  
    end = time.time()
    matchingTime += (end - st)
    if termination:
            break

    nbS += offsetB
    naS += offsetA

print("blocking time (in mins)", blockingTime / 60)
print("matching time (in mins)", matchingTime / 60)
if tp + fp > 0:
    print("TP=", tp, "Recall=", tp / TP, "Precision=", tp / (tp + fp), "pairsNo=", pairsNo)

blocking time (in mins) 0.6470078786214193
matching time (in mins) 3.489612682660421
TP= 12424 Recall= 0.30381727924094587 Precision= 0.3356567785162371 pairsNo= 61566


In [ ]:
# IDEIA, TODA VEZ QUE FOR FAZER UM MATCHING, VERIFICAR SE JA BLOCOU UM SEXO DIFERENTE ANTERIOR. MANTER UMA DIFERENÇA 

# BackUp


In [35]:
df2["gender_code"].value_counts()

gender_code
F    21008
M    17886
U     1998
Name: count, dtype: int64

In [ ]:
ids = [('BZ58220', 'F'), ('BZ5995', 'M'), ('BZ75127', 'M'), ('BZ71367', 'F'), ('BZ75022', 'F'), ('BZ75023', 'F'), ('BZ39687', 'M'), ('BZ57774', 'M'), ('EH487995', 'F')]
ids_igual = [0, 0, 0, 0, 0, 0, 0, 0, 0]
adiciona_valor_bloco(ids, ids_igual, "M", 1, "4124")
ids

[('BZ58220', 'F'),
 ('BZ5995', 'M'),
 ('BZ75127', 'M'),
 ('BZ71367', 'F'),
 ('BZ75022', 'F'),
 ('BZ75023', 'F'),
 ('BZ39687', 'M'),
 ('BZ57774', 'M'),
 ('EH487995', 'F'),
 ('4124', 'M')]